## Generating data for ML

In [1]:
import pandas as pd
from collections import Counter
import random 
import ultimate
import numpy as np
# define card set
suits = ['Hearts', 'Diamonds', 'Clubs', 'Spades']
ranks = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']
rank_values = {rank: i for i, rank in enumerate(ranks, start=2)}

deck = [{'rank': rank, 'suit': suit} for suit in suits for rank in ranks]

combinations = ["High Card", "One Pair", "Two Pair", "Three of a Kind", "Four of a Kind", 
                "Full House", "Straight", "Flush", "Straight Flush", "Royal Flush"]
combinations_values = {combination: i for i, combination in enumerate(combinations, start=1)}
# set ordered winning combinations
winning_hands = ["High Card", "One Pair", "Two Pair", "Three of a Kind", "Straight", "Flush", 
                "Full House", "Four of a Kind", "Straight Flush", "Royal Flush"]

winning_hand_ranks = {hand: i for i, hand in enumerate(winning_hands)}
#enumerate the deck
enumerated_deck = dict(enumerate(deck, start=1))
num_deck = list(range(1, 53))

In [2]:
# first round options
print(52*51*50*49 / 4)
# second round options
print(52*51*50*49*48*47*46 / 12)
# third round options
print(52*51*50*49*48*47*46*45*44 / 24)
enumerated_deck

1624350.0
56189515200.0
55627620048000.0


{1: {'rank': '2', 'suit': 'Hearts'},
 2: {'rank': '3', 'suit': 'Hearts'},
 3: {'rank': '4', 'suit': 'Hearts'},
 4: {'rank': '5', 'suit': 'Hearts'},
 5: {'rank': '6', 'suit': 'Hearts'},
 6: {'rank': '7', 'suit': 'Hearts'},
 7: {'rank': '8', 'suit': 'Hearts'},
 8: {'rank': '9', 'suit': 'Hearts'},
 9: {'rank': '10', 'suit': 'Hearts'},
 10: {'rank': 'J', 'suit': 'Hearts'},
 11: {'rank': 'Q', 'suit': 'Hearts'},
 12: {'rank': 'K', 'suit': 'Hearts'},
 13: {'rank': 'A', 'suit': 'Hearts'},
 14: {'rank': '2', 'suit': 'Diamonds'},
 15: {'rank': '3', 'suit': 'Diamonds'},
 16: {'rank': '4', 'suit': 'Diamonds'},
 17: {'rank': '5', 'suit': 'Diamonds'},
 18: {'rank': '6', 'suit': 'Diamonds'},
 19: {'rank': '7', 'suit': 'Diamonds'},
 20: {'rank': '8', 'suit': 'Diamonds'},
 21: {'rank': '9', 'suit': 'Diamonds'},
 22: {'rank': '10', 'suit': 'Diamonds'},
 23: {'rank': 'J', 'suit': 'Diamonds'},
 24: {'rank': 'Q', 'suit': 'Diamonds'},
 25: {'rank': 'K', 'suit': 'Diamonds'},
 26: {'rank': 'A', 'suit': 'Diamo

## For one player

In [3]:
# ante is 1, blind is 1, player bets 1 or 0 (return bet or 
# fold depending whether you will win or lose)
fast_deck = np.array([enumerated_deck[i] for i in range(1,53)])
def decide_game_victor_and_calculate_rewards_for_bets(row):
    
    #player_hand = [enumerated_deck[card] for card in row[0:7]]
    #dealer_hand = [enumerated_deck[card] for card in row[2:]]
    row = np.asarray(row, dtype=int)
    player_hand = list(fast_deck[row[0:7] - 1])
    dealer_hand = list(fast_deck[row[2:] - 1])

    player_combination = ultimate.get_best_hand(player_hand)
    dealer_combination = ultimate.get_best_hand(dealer_hand)

    player_rank = winning_hand_ranks[player_combination]
    dealer_rank = winning_hand_ranks[dealer_combination]

    victor = 0 # 0 = dealer, 1 = player
    
    if player_rank > dealer_rank:
        victor = 1
    elif player_rank == dealer_rank:
        result = ultimate.decider(player_combination, player_hand, 
                                  dealer_combination, dealer_hand)
        if result == "player":
            victor = 1
        elif result == "dealer":	
            victor = 0
        else:
            victor = random.randint(0, 1)	# need to decide about this
    else:
        victor = 0

    # check if ante is valid
    dealer_has_something = ultimate.dealer_has_pair_or_better(dealer_hand[:2], dealer_hand[2:])
    blind_won = ultimate.has_blind(1, player_combination) - 1 #how much blind got us

    # calculate rewards for first and second rounds (in third victory is already bet, defeat is fold)
    first_round = 0
    second_round = 0
    third_round = 0

    # if its a draw, we leave both values at zero
    if victor == 1:
        first_round = 4 + blind_won + (1 if dealer_has_something else 0)
        second_round = 2 + blind_won + (1 if dealer_has_something else 0)
        third_round = 1 + blind_won + (1 if dealer_has_something else 0)
    elif victor == 0:
        first_round = -6 + (1 if not dealer_has_something else 0)
        second_round = -4 + (1 if not dealer_has_something else 0)
        third_round = -3 + (1 if not dealer_has_something else 0)
    
    return [third_round, first_round, second_round]

# function for conversion back to cards for checking
def convert_to_cards(row):
    cards = row[:9]
    output = [(enumerated_deck[card]["rank"], enumerated_deck[card]["suit"])  for card in cards]
    return output


In [4]:
# full dataset
import itertools
deck = np.arange(1, 53)
all_starting_hands = list(itertools.combinations(deck, 2))  # 1326 hands

multiplier_1 = 15 # flops per hand
multiplier_2 = 15 # rivers per flop
multiplier_D = 20 # dealers per river

num_hands = len(all_starting_hands) * multiplier_1 * multiplier_2 * multiplier_D  # total combinations (e.g., 9.9 million)
rows_array = np.empty((num_hands, 9), dtype=np.uint8)  # 9 = C1, C2, R1–R5, D1, D2
header = ["C1", "C2", "R1", "R2", "R3", "R4", "R5", "D1", "D2"]
index = 0
for player_hand in all_starting_hands:
    player_hand = np.array(player_hand)
    
    # Precompute available cards once per player hand
    available_after_player = np.setdiff1d(deck, player_hand, assume_unique=True)

    for _ in range(multiplier_1):  # flops
        flop = np.random.choice(available_after_player, size=3, replace=False)
        used_flop = np.concatenate([player_hand, flop])
        available_after_flop = np.setdiff1d(deck, used_flop, assume_unique=True)

        for _ in range(multiplier_2):  # rivers
            river = np.random.choice(available_after_flop, size=2, replace=False)
            used_river = np.concatenate([used_flop, river])
            available_after_river = np.setdiff1d(deck, used_river, assume_unique=True)

            for _ in range(multiplier_D):  # dealers
                dealer = np.random.choice(available_after_river, size=2, replace=False)
                full_row = np.concatenate([player_hand, flop, river, dealer])
                rows_array[index] = full_row
                index += 1
full_data_set = pd.DataFrame(rows_array, columns=header)
print("generated variables")
full_data_set[["Q3", "Q1", "Q2"]] = full_data_set.apply(decide_game_victor_and_calculate_rewards_for_bets, axis=1, result_type="expand")
print("calculated bets")
full_data_set["r2ID"] = full_data_set.index // (multiplier_2 * multiplier_D)
full_data_set["r3ID"] = full_data_set.index // multiplier_D
exp_values_Q1 = full_data_set.groupby(["C1", "C2"], sort=False).mean().reset_index().drop(columns=["R1", "R2", "R3", "R4", "R5", "D1", "D2", "Q3", "Q2", "r2ID", "r3ID"])
exp_values_Q2 = full_data_set.groupby(["C1", "C2", "R1", "R2", "R3", "r2ID"], sort=False).mean().reset_index().drop(columns=["R4", "R5", "D1", "D2", "Q3", "Q1", "r2ID", "r3ID"])
exp_values_Q3 = full_data_set.groupby(["C1", "C2", "R1", "R2", "R3", "R4", "R5", "r3ID"], sort=False).mean().reset_index().drop(columns=["D1", "D2", "Q2", "Q1", "r2ID", "r3ID"])
print("mean")

data_for_first_round = exp_values_Q1#.drop(columns=["Q1"])
data_for_second_round = exp_values_Q2#.drop(columns=["Q2"])
data_for_third_round = exp_values_Q3#.drop(columns=["Q3"])


#testing
pd.set_option('display.max_rows', 20)  # Show up to 100 rows

np.save("data_for_first_round.npy", data_for_first_round.to_numpy())
np.save("data_for_second_round.npy", data_for_second_round.to_numpy())
np.save("data_for_third_round.npy", data_for_third_round.to_numpy())

generated variables
calculated bets
mean


In [5]:
print(data_for_first_round)

      C1  C2        Q1
0      1   2 -2.649778
1      1   3 -0.522778
2      1   4 -0.503889
3      1   5 -2.506000
4      1   6 -1.401778
...   ..  ..       ...
1321  49  51  1.865000
1322  49  52  0.711000
1323  50  51  1.074444
1324  50  52  1.143889
1325  51  52  1.250778

[1326 rows x 3 columns]


## For n players

In [6]:
# ante is 1, blind is 1, player bets 1 or 0 (return bet or 
# fold depending whether you will win or lose)
def decide_game_victor_and_calculate_rewards_for_bets_n(row):
    
    # Cache row lookup just once
    cards = row[["C1_P1", "C2_P1", "R1", "R2", "R3", "R4", "R5", "D1", "D2"]].to_numpy()

    # Fast deck lookup in bulk (vectorized access)
    card_objs = [enumerated_deck[card] for card in cards]

    # Build hands using slicing
    player_hand = card_objs[0:2] + card_objs[2:7]  # C1, C2 + R1–R5
    dealer_hand = card_objs[2:7] + card_objs[7:9]

    
    player_combination = ultimate.get_best_hand(player_hand)
    dealer_combination = ultimate.get_best_hand(dealer_hand)

    player_rank = winning_hand_ranks[player_combination]
    dealer_rank = winning_hand_ranks[dealer_combination]

    victor = 0 # 0 = dealer, 1 = player
    
    if player_rank > dealer_rank:
        victor = 1
    elif player_rank == dealer_rank:
        result = ultimate.decider(player_combination, player_hand, 
                                  dealer_combination, dealer_hand)
        if result == "player":
            victor = 1
        elif result == "dealer":	
            victor = 0
        else:
            victor = random.randint(0, 1)	# need to decide about this
    else:
        victor = 0

    # check if ante is valid
    dealer_has_something = ultimate.dealer_has_pair_or_better(dealer_hand[:2], dealer_hand[2:])
    blind_won = ultimate.has_blind(1, player_combination) - 1 #how much blind got us

    # calculate rewards for first and second rounds (in third victory is already bet, defeat is fold)
    first_round = 0
    second_round = 0
    third_round = 0

    # if its a draw, we leave both values at zero
    if victor == 1:
        first_round = 4 + blind_won + (1 if dealer_has_something else 0)
        second_round = 2 + blind_won + (1 if dealer_has_something else 0)
        third_round = 1 + blind_won + (1 if dealer_has_something else 0)
    elif victor == 0:
        first_round = -6 + (1 if not dealer_has_something else 0)
        second_round = -4 + (1 if not dealer_has_something else 0)
        third_round = -3 + (1 if not dealer_has_something else 0)
    
    return [third_round, first_round, second_round]

# function for conversion back to cards for checking
def convert_to_cards_n(row, n):
    cards = row[:(5 + 2 * n)]
    output = [(enumerated_deck[card]["rank"], enumerated_deck[card]["suit"])  for card in cards]
    return output


In [7]:
import random
import numpy as np
import pandas as pd
import itertools
deck = np.arange(1, 53)
all_starting_hands = list(itertools.combinations(deck, 2))  # 1326 hands

multiplier_1 = 15  # flops per hand
multiplier_2 = 15 # rivers per flop
multiplier_D = 20 # dealers per river
players = 4 # num of players

num_hands = len(all_starting_hands) * multiplier_1 * multiplier_2 * multiplier_D  # total combinations (e.g., 9.9 million)
rows_array = np.empty((num_hands, 7 + 2 * players), dtype=np.uint8)  # 9 = C1, C2, R1–R5, D1, D2
header = [] 
playersHeader = []  # Cards j Player i
for i in range(players):
    playersHeader += [f"C1_P{i+1}", f"C2_P{i+1}"]
header += playersHeader
header += ["R1", "R2", "R3", "R4", "R5", "D1", "D2"]
print(header)
index = 0
for player_hand in all_starting_hands:
    player_hand = np.array(player_hand)
    
    # Precompute available cards once per player hand
    available_after_player = np.setdiff1d(deck, player_hand, assume_unique=True)

    other_players = np.random.choice(available_after_player, size = players * 2 - 2, replace=False)
    players_hand = np.concatenate([player_hand, other_players])
    available_after_players = np.setdiff1d(deck, players_hand, assume_unique=True)
    for _ in range(multiplier_1):  # flops
        flop = np.random.choice(available_after_players, size=3, replace=False)
        used_flop = np.concatenate([players_hand, flop])
        available_after_flop = np.setdiff1d(deck, used_flop, assume_unique=True)

        for _ in range(multiplier_2):  # rivers
            river = np.random.choice(available_after_flop, size=2, replace=False)
            used_river = np.concatenate([used_flop, river])
            available_after_river = np.setdiff1d(deck, used_river, assume_unique=True)

            for _ in range(multiplier_D):  # dealers
                dealer = np.random.choice(available_after_river, size=2, replace=False)
                full_row = np.concatenate([players_hand, flop, river, dealer])
                rows_array[index] = full_row
                index += 1
full_data_set = pd.DataFrame(rows_array, columns=header)

print("generated variables")
full_data_set[["Q3", "Q1", "Q2"]] = full_data_set.apply(
    decide_game_victor_and_calculate_rewards_for_bets_n, axis=1, result_type="expand"
)
print("rewards calculated")

q1_keys = playersHeader
q2_keys = playersHeader + ["R1", "R2", "R3"]
q3_keys = playersHeader + ["R1", "R2", "R3", "R4", "R5"]

full_data_set["r2ID"] = full_data_set.index // (multiplier_2 * multiplier_D)
full_data_set["r3ID"] = full_data_set.index // multiplier_D


exp_values_Q1 = (
    full_data_set.groupby(q1_keys, sort=False).mean(numeric_only=True).reset_index()
    .drop(columns=[c for c in ["R1","R2","R3","R4","R5","D1","D2","Q3","Q2", "r2ID", "r3ID"] if c in full_data_set.columns])
)
exp_values_Q2 = (
    full_data_set.groupby(q2_keys + ["r2ID"], sort=False).mean(numeric_only=True).reset_index()
    .drop(columns=[c for c in ["R4","R5","D1","D2","Q3","Q1", "r2ID", "r3ID"] if c in full_data_set.columns])
)
exp_values_Q3 = (
    full_data_set.groupby(q3_keys + ["r3ID"], sort=False).mean(numeric_only=True).reset_index()
    .drop(columns=[c for c in ["D1","D2","Q2","Q1", "r2ID", "r3ID"] if c in full_data_set.columns])
)

print("mean")

#exp_values_Q1["Y1"] = (exp_values_Q1["Q1"] > 0).astype(int)
#exp_values_Q2["Y2"] = (exp_values_Q2["Q2"] > 0).astype(int)
#exp_values_Q3["Y3"] = (exp_values_Q3["Q3"] > 0).astype(int)
#print("y decided")

data_for_first_round  = exp_values_Q1#.drop(columns=["Q1"])
data_for_second_round = exp_values_Q2#.drop(columns=["Q2"])
data_for_third_round  = exp_values_Q3#.drop(columns=["Q3"])

pd.set_option('display.max_rows', 30)  # Show up to 20 rows

np.save("data_for_first_round_n.npy",  data_for_first_round.to_numpy())
np.save("data_for_second_round_n.npy", data_for_second_round.to_numpy())
np.save("data_for_third_round_n.npy",  data_for_third_round.to_numpy())


['C1_P1', 'C2_P1', 'C1_P2', 'C2_P2', 'C1_P3', 'C2_P3', 'C1_P4', 'C2_P4', 'R1', 'R2', 'R3', 'R4', 'R5', 'D1', 'D2']
generated variables
rewards calculated
mean


In [8]:
print(convert_to_cards_n(data_for_third_round.iloc[3].values, 2))

[('2', 'Hearts'), ('3', 'Hearts'), ('5', 'Clubs'), ('2', 'Clubs'), ('5', 'Diamonds'), ('10', 'Spades'), ('5', 'Hearts'), ('7', 'Diamonds'), ('8', 'Diamonds')]
